# PREDICT DATA

In [1]:
#r "nuget:Microsoft.ML,1.5.2"
#r "nuget:Microsoft.ML.LightGBM,1.5.2"
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers.LightGbm;

MLContext mlContext = new MLContext(seed: 1);

In [1]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Models.csx" 

const string DATASET_PATH = "./sensors_data.csv";
IDataView data = mlContext.Data.LoadFromTextFile<ModelInput>(
    path: DATASET_PATH,
    hasHeader: true,
    separatorChar: ',');

var shuffledData = mlContext.Data.ShuffleRows(data, seed: 1);
var split = mlContext.Data.TrainTestSplit(shuffledData, testFraction: 0.3);
var trainingData = split.TrainSet;
var testingData = split.TestSet; 

In [1]:
var featureColumns = new[] { "Temperature", "Luminosity", "Infrared", "Distance"};

var trainingPipeline = mlContext.Transforms.Conversion.MapValueToKey("Label")
    .Append(mlContext.Transforms.Concatenate("Features", featureColumns))
    //.Append(mlContext.Transforms.NormalizeMinMax("Features")) // https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.trainers.lightgbm.lightgbmmulticlasstrainer?view=ml-dotnet
    .Append(mlContext.MulticlassClassification.Trainers.LightGbm("Label", "Features"))
    .Append(mlContext.Transforms.Conversion.MapKeyToValue("PredictedLabel"));

var model = trainingPipeline.Fit(trainingData);

### Evaluate the model 

In [1]:
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Formatters.csx"
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.Data.Analysis;

Formatters.Categories = new string[] { "FlashLight", "Infrared", "DayLight", "Lighter" };
Formatters.Register<ConfusionMatrix>();
Formatters.Register<MulticlassClassificationMetrics>();

var predictions = model.Transform(testingData);
var metrics = mlContext.MulticlassClassification.Evaluate(predictions, "Label", "Score", "PredictedLabel");

display(metrics);

display(metrics.ConfusionMatrix);

display(p[@style:"font-size: 40px"]($"MicroAccuracy: {metrics.MicroAccuracy:P2}, MacroAccuracy: {metrics.MacroAccuracy:P2}"));

EVALUATION: multi-class classification Class Value Note MacroAccuracy 0.918 the closer to 1, the better MicroAccuracy 0.923 the closer to 1, the better LogLoss 0.323 the closer to 0, the better LogLoss per Class FlashLight 0.353 the closer to 0, the better Infrared 0.620 the closer to 0, the better DayLight 0.194 the closer to 0, the better Lighter 0.213 the closer to 0, the better

Confusion Matrix Predicted FlashLight Infrared DayLight Lighter Recall Truth FlashLight 70 1 2 2 0.9333 Infrared 0 52 6 2 0.8667 DayLight 0 1 103 5 0.945 Lighter 1 0 3 51 0.9273 Precision 0.9859 0.963 0.9035 0.85 total = 299

MicroAccuracy: 92.31%, MacroAccuracy: 91.81%

## Predict data

In [1]:
#r "nuget:MathNet.Numerics"
#load "C:\Users\dcost\source\repos\SmartFireAlarm\SmartFireAlarm\Jupyter\Helpers.csx"

var sampleData = new ModelInput
{
    Temperature = 32F,
    Luminosity = 22F,
    Infrared = 0F,
    Distance = 20F,
    PIR = 1F,
    Humidity = 13F,
    CreatedAt = "01/03/2020 10:22:08"
};

var predictor = mlContext.Model.CreatePredictionEngine<ModelInput, ModelOutput>(model);
var predicted = predictor.Predict(sampleData);

display(p[@style:"font-size: 40px"]($"{predicted.PredictedLabel}"));

display(Helpers.GetPredictionPerClass(predicted, predictor.OutputSchema));

FlashLight

0.9320,FlashLight
0.0625,DayLight
0.0051,Infrared
0.0003,Lighter
